# Testing

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [53]:
# Try fetching from first anime from the first page of top anime
nth_anime = 70 - 1# Indexing start at 0
page = nth_anime - nth_anime%50
nth_anime = nth_anime%50
response = requests.get(f'https://myanimelist.net/topanime.php?limit={page}')
soup = BeautifulSoup(response.text, 'html.parser')
tags = soup.find_all('tr', {"class": "ranking-list"})
url = BeautifulSoup(str(tags[nth_anime]), 'html.parser').find('a')['href']
print(url)

https://myanimelist.net/anime/52991/Sousou_no_Frieren


In [35]:
# Get information about that anime
sub_rq = requests.get(url)
soup = BeautifulSoup(sub_rq.text, 'html.parser')

out_dict = {}

table_tags_list = soup.find_all(attrs={"class":'spaceit_pad'})


for i in table_tags_list:
    s:str = i.text
    s = s.replace('\n','')
    
    s = s.split(':')
    if len(s) == 2:
        out_dict[s[0]] = s[1]

out_dict["description"] = soup.find(attrs={"itemprop":'description'}).text
out_dict['Link'] = url
out_dict

{'Synonyms': ' Frieren at the Funeral  ',
 'Japanese': ' 葬送のフリーレン  ',
 'Type': 'TV',
 'Episodes': '  28  ',
 'Status': '  Finished Airing  ',
 'Aired': '  Sep 29, 2023 to Mar 22, 2024  ',
 'Premiered': 'Fall 2023',
 'Producers': 'Aniplex,       Dentsu,       Shogakukan-Shueisha Productions,       Nippon Television Network,       TOHO animation,       Shogakukan ',
 'Licensors': '    None found, add some ',
 'Studios': 'Madhouse ',
 'Source': '      Manga    ',
 'Genres': 'AdventureAdventure,         DramaDrama,         FantasyFantasy ',
 'Demographic': 'ShounenShounen ',
 'Duration': '  24 min. per ep.  ',
 'Rating': '  PG-13 - Teens 13 or older  ',
 'Score': '9.341 (scored by 469934469,934 users)      1          indicates a weighted score.    ',
 'Ranked': "  #122    based on the top anime page. Please note that 'Not yet aired' and 'R18+' titles are excluded.    ",
 'Popularity': '  #224',
 'Members': '    830,523',
 'Favorites': '  48,741',
 'description': 'During their decade-long q

# Scraping

In [53]:
import time
import random
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import os
import pickle
import logging
import sys
from tqdm.notebook import tqdm

### Define functions

In [26]:
# Get a random user-agent
def get_random_user_agent():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    ]
    return random.choice(user_agents)

# Make a request with retries
def make_request(url):
    headers = {'User-Agent': get_random_user_agent()}
    session = requests.Session()

    retry = Retry(
        total=5, # Total number of retries
        backoff_factor=1, # Exponential backoff factor
        status_forcelist=[500, 502, 503, 504], # Retry on these HTTP status codes
        method_whitelist=["HEAD", "GET", "OPTIONS"] # Retry on these HTTP methods
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    try:
        response = session.get(url, headers=headers)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx and 5xx)
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    
# Create logger
def get_logger(logname, log_level=logging.INFO):
    # If log file doesn't exist, create new one
    if not os.path.exists(logname):
        logger = logging.getLogger()
        fhandler = logging.FileHandler(filename=logname, encoding="utf-8")
        formatter = logging.Formatter('%(asctime)s [%(levelname)s] > %(message)s')
        fhandler.setFormatter(formatter)
        logger.addHandler(fhandler)
        logger.setLevel(log_level)
        print("Created log " + logname)

    # Else use the existing file
    else:
        # Configure the logger
        logging.basicConfig(
            filename= logname,    # Use the existing log file
            filemode='a',          # Append mode, to add to the existing log file. 'w' for overwrite
            format='%(asctime)s [%(levelname)s] - %(name)s:%(filename)s > %(message)s', # Format using %() for backward compatibility
            level=log_level    # Set the logging level as needed. DEBUG means log everything above DEBUG
        )

        # Create a logger object
        logger = logging.getLogger()
        print("Loaded log " + logname)

    return logger

### Use pickle to store the information so we can continue the collection later

In [58]:
# Pickle the list so we can continue from where the loop last fail
pname = "Data/out_dict_list.pkl" # Pickle Filenamae

# Check if pickle file exist
if not os.path.exists(pname):
    # Create file
    with open(pname, 'wb') as f:
        pickle.dump([], f)

else:
    print("File already exist")

# Load from a file
with open(pname, 'rb') as f:
    out_dict_list = pickle.load(f)

last_page = (len(out_dict_list)//50)*50
print(len(out_dict_list))

File already exist
16765


### Get logger so we can log the data collection

In [37]:
logger = get_logger("AnimeScrape.log")

Loaded log AnimeScrape.log


### Scrape MyAnimeList

In [54]:
verbose = False # If true print the progress
end_at = 17000

# Progress Bar
pbar = tqdm(total=end_at - last_page)

# Main scraping loop
for page in range(last_page, end_at, 50):
    try:
        # Fetch the page with the list of top anime
        response = make_request(f'https://myanimelist.net/topanime.php?limit={page}')
        
        # Check if the request was successful
        if response is None:
            break

        # Parse the page content
        soup = BeautifulSoup(response, 'html.parser')
        tags = soup.find_all('tr', {"class": "ranking-list"})
        
        for i, tag in enumerate(tags):
            try:
                # Extract the URL of the anime detail page
                url = tag.find('a')['href']
                
                # Fetch the anime detail page
                sub_response = make_request(url)
                if sub_response is None:
                    continue
                
                # Parse the detail page content
                sub_soup = BeautifulSoup(sub_response, 'html.parser')
                
                # Find all relevant tags
                table_tags_list = sub_soup.find_all(attrs={"class": 'spaceit_pad'})
                
                out_dict = {}
                for j in table_tags_list:
                    s: str = j.text.replace('\n', '')
                    s_split: list[str] = s.split(':')

                    # Check if the split was successful and store the key-value pair
                    if len(s_split) == 2:
                        out_dict[s_split[0].strip()] = s_split[1].strip()
                        
                
                out_dict['url'] = url
                #######
                #Extra Features:
                    # Extract the description
                description_tag = sub_soup.find(attrs={"itemprop": 'description'})
                out_dict["description"] = description_tag.text if description_tag else ""
                    # Name
                out_dict['Name'] = sub_soup.find(attrs={"class": 'title-name h1_bold_none'}).text
                if not (sub_soup.find(attrs={"class": 'title-english title-inherit'}) is None):
                    out_dict['English Name'] = sub_soup.find(attrs={"class": 'title-english title-inherit'}).text
                    # Reviews
                        # Recommended
                rev_R = sub_soup.find(attrs={"class": 'recommended'})
                out_dict['Recommended'] =rev_R.text.replace('\n','')
                        # MixedFeelings
                rev_M = sub_soup.find(attrs={"class": 'mixed-feelings'})
                out_dict['Mixed Feelings'] =rev_M.text.replace('\n','')
                        # NotRecommended
                rev_M = sub_soup.find(attrs={"class": 'not-recommended'})
                out_dict['Not Recommended'] =rev_M.text.replace('\n','')
                
                # Log the progress and print progress information
                msg = f"Page {page}, Target {i}, Name: {out_dict.get('Name', 'N/A')}"
                if verbose:
                    print(msg)
                logger.info(msg)
                
                # Add the dictionary to the list
                out_dict_list.append(out_dict)
                
                pbar.update(1)
                # Introduce a random delay between requests
                time.sleep(random.uniform(0.5, 2)) # Delay between 0.5 to 2 seconds
            except Exception as e:
                msg = f"Failed to process target {i} on page {page}: {e}"
                print(msg)
                logger.error(msg)
        
    except Exception as e:
        with open(pname, 'wb') as f:
            pickle.dump(out_dict_list, f)
        msg = f"Failed to process page {page}: {e}"
        print(msg)
        logger.error(msg)

        break
    

  0%|          | 0/250 [00:00<?, ?it/s]

C:\Users\nadzm\AppData\Local\Temp\ipykernel_25152\755529457.py:15: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  retry = Retry(


KeyboardInterrupt: 

## Turn into DataFrame and Save as .csv

In [64]:
df = pd.DataFrame(out_dict_list)
df.to_csv("Data/AnimeRaw.csv")